In [164]:
import math
from scripts_stock.cfg.out_file_name import OutFileName
from scripts_stock.cfg.set_dir import ProjectDir
from scripts_stock.cfg.stock_list import *
from scripts_stock.utils.analysis.stock_stat_index import df_to_stock_df, stock_kdj
import os
from scripts_stock.utils.common import CommonScript
from scripts_stock.utils.string_process import StringProcess


def get_data_sql_str(table):
    return f"""SELECT *  FROM {table} limit 10 """

def get_data_sql_str_macd(stock_index):
    return f"""SELECT *  FROM t_stock_kdj_daily_all where date>='2022-9-23' and stock_index={stock_index} """

def get_train_data():
    aa = f"""
    select t1.*,t2.diff_5,t2.diff_3_cate,t2.diff_5_cate,t2.diff_7_cate
    from t_stock_kdj_daily_all AS t1
    join t_stock_return_fuquan AS t2
    on t1.date=t2.date and t1.stock_index=t2.stock_index
    where close<8 and t2.date<='2024-09-01' and t2.date>='2023-01-01'
    """
    return aa

def get_predict_data():
    aa = f"""
    select t1.*,t2.diff_5,t2.diff_3_cate,t2.diff_5_cate,t2.diff_7_cate
    from t_stock_kdj_daily_all AS t1
    join t_stock_return_fuquan AS t2
    on t1.date=t2.date and t1.stock_index=t2.stock_index
    where close<8 and t2.date>='2024-09-01'
    """
    return aa

In [165]:
conn = CommonScript.connect_to_db("test.db")
cursor = conn.cursor()
df1 = pd.read_sql_query(get_train_data(),conn)
# df1 = df1[df1["stock_index"]==601398]
df1["boll_diff"]= df1["boll_ub"] - df1["boll_lb"]
#df1.columns
features = [ 'macdh', 'cci', 'rsi_6', 'rsi_12', 'rsi_24', 'kdjk',
       'kdjd', 'kdjj', 'boll_ub', 'boll_lb','boll_diff', 'macd', 'macds', 'wr_6', 'wr_10']
df_pred_y = ['diff_5_cate']

df2 = df1[features+df_pred_y].dropna()
#print(df1.head(5))
df2.groupby(df_pred_y[0]).count()

,macdh,cci,rsi_6,rsi_12,rsi_24,kdjk,kdjd,kdjj,boll_ub,boll_lb,boll_diff,macd,macds,wr_6,wr_10
diff_5_cate,,,,,,,,,,,,,,,
0.0,13673,13673,13673,13673,13673,13673,13673,13673,13673,13673,13673,13673,13673,13673,13673
1.0,12722,12722,12722,12722,12722,12722,12722,12722,12722,12722,12722,12722,12722,12722,12722


In [166]:
from sklearn import tree#导入模块
from sklearn.model_selection import train_test_split
X = df2[features]
Y = df2[df_pred_y[0]]
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

In [167]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import torch.utils.data



In [168]:
# # Define parameters
# step_size = 4
# N = 800
# forecast_start = 720

# # Generate data
# t = np.arange(0, N)
# x = np.sin(0.02 * t) + 2 * np.random.rand(N)
# df = pd.DataFrame(x)

# # Plot data
# plt.plot(df)
# plt.axvline(df.index[forecast_start], c="r", label="forecast start point")
# plt.legend()
# plt.show()

In [169]:
def create_labels(data, step):
    X = np.array([data[i:i+step] for i in range(len(data) - step)])
    y = np.array(data[step:])
    return X, y

In [170]:
# # Prepare data for training and testing
# values = df.values
# train, test = values[:forecast_start, :], values[forecast_start:N, :]

# # generate sequence data
# trainX, trainY = create_labels(train, step_size)
# testX, testY = create_labels(test, step_size)

# # Reshape data for LSTM input
# trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
# testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

# trainY.shape

In [171]:
aa = x_train.values

trainX = np.reshape(x_train.values, (x_train.values.shape[0], 1, x_train.values.shape[1]))
trainY = y_train.values.reshape(y_train.values.shape[0],1)
trainY.shape

testX = np.reshape(x_test.values, (x_test.values.shape[0], 1, x_test.values.shape[1]))
testY = y_test.values.reshape(y_test.values.shape[0],1)



In [172]:

## stock data
# aa = x_train.values

# trainX = np.reshape(aa, (aa.shape[0], 1, aa.shape[1]))
# trainY = y_train.values.reshape(156942,1).shape


# Convert data to PyTorch tensors
trainX_tens = torch.tensor(trainX, dtype=torch.float32)
trainY_tens = torch.tensor(trainY, dtype=torch.float32)
testX_tens = torch.tensor(testX, dtype=torch.float32)
testY_tens = torch.tensor(testY, dtype=torch.float32)

# Create DataLoader for training
train_dataset = torch.utils.data.TensorDataset(trainX_tens, trainY_tens)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64)

In [173]:
# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Take the last time step's output
        return out
 

In [174]:
# Hyperparameters
input_size = 15
hidden_size = 128
output_size = 1
epochs = 100
learning_rate = 0.01
 
# Instantiate LSTM model
model = LSTMModel(input_size=input_size, hidden_size=hidden_size, output_size=output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [175]:
# Train the model
for epoch in range(epochs):
    model.train()
    for batch_X, batch_Y in train_loader:
        optimizer.zero_grad()  # Clears the gradients of all optimized parameters.
        output = model(batch_X)

        # Computes the loss between the model predictions and the ground
        # truth labels for the current mini-batch.
        loss = criterion(output, batch_Y)

        # Computes gradients of the loss with respect to model parameters.
        loss.backward()

        # Updates model parameters based on the computed gradients using
        # the specified optimization algorithm.
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 0.2581
Epoch [20/100], Loss: 0.2535
Epoch [30/100], Loss: 0.2553
Epoch [40/100], Loss: 0.2555
Epoch [50/100], Loss: 0.2533
Epoch [60/100], Loss: 0.2540
Epoch [70/100], Loss: 0.2541
Epoch [80/100], Loss: 0.2574
Epoch [90/100], Loss: 0.2590
Epoch [100/100], Loss: 0.2620


In [176]:
with torch.no_grad():
    a1 = model(trainX_tens).numpy()
a2 = [x[0] for x in a1]
a3 = [1 if x>0.5 else 0 for x in a2 ]
from sklearn.metrics import accuracy_score
print(accuracy_score(y_train,a3))


0.4836143208941087
